In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='jshtml')
import seaborn as sns

#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=2.2)

## Problema 1 (Mola)

<font size=3px>Uma mola de constante elástica $k$ está presa a um bloco de massa $m$. O bloco é
    colocado na posição $x_0$ e solto a partir do repouso. A mola encontra-se relaxada quando a posição do bloco
    é $x=0$. Se não existe atrito com o plano, determine:<br>
    a) a velocidade máxima do bloco,<br>
    b) a posição máxima do bloco,<br>
    c) a posição máxima do bloco se a superfície possui um coeficiente de atrito cinético $\mu_k$.
</font>

In [ ]:
L0 = 0.
k = 1.
m = 1.
muk = 0.
g=9.8

def F(t,xv):
    x,v = xv
    if xv[1] != 0.:
        return -k*(x-L0) -muk*m*g*xv[1]/abs(xv[1])
    else:
        return -k*(x-L0)

def a(t,xv):
    return F(t,xv)/m

def RHS(t,xv):
    return [xv[1],a(t,xv)]

v0 = 0.
x0 = -1.
xv0 = [x0,v0]
t0, tf = 0.,15.
sol = scipy.integrate.solve_ivp(RHS,(t0,tf),xv0,rtol=1e-7,atol=1e-7)
xF = scipy.interpolate.interp1d(sol.t,sol.y[0,:])
vF = scipy.interpolate.interp1d(sol.t,sol.y[1,:])

def KF(t):
    return (m*vF(t)**2)/2.

def UF(t):
    return k*((xF(t)-L0)**2)/2.

In [ ]:
tmin, tmax = t0,tf
dt = 0.1
tpts = np.linspace(tmin,tmax,100)

fig = plt.figure(figsize=(22,7))
ax1 = plt.subplot(121)
pt, = ax1.plot([xF(tmin),0.],'o',markersize=25)
line, = ax1.plot([0.,xF(tmin)],[0.,0.],'--')
xlabel = ax1.set_xlabel('x (m)')
ax1.set_xlim(-2*abs(x0-L0),2*abs(x0-L0))
ax1.set_ylim(-0.1,0.1)

ax2 = plt.subplot(122)
Kl, = ax2.plot(tpts,KF(tpts),'--',linewidth=4.0)
Ul, = ax2.plot(tpts,UF(tpts),'--',linewidth=4.0)
El, = ax2.plot(tpts,KF(tpts)+UF(tpts),linewidth=4.0)
xlabel = ax2.set_xlabel('t (s)')
ax2.legend(['K','U','K+U'],loc='upper right',framealpha=1.0,fontsize=25.0)


def init():
    tpts = [tmin]
    pt.set_data([xF(tmin)],[0.])
    line.set_data([0.,xF(tmin)],[0.,0.])
    line._linewidth = 3.0
    Kl.set_data(tpts,KF(tpts))
    Ul.set_data(tpts,UF(tpts))    
    return (pt,line,Kl,Ul)

def animate(i):
    
    tA = tmin+i*dt
    tpts = np.linspace(tmin,tA,100)
    xpt = [xF(tA)]
    ypt = [0.]
    pt.set_data(xpt,ypt)
    line.set_data([0.,xF(tA)],[0.,0.])
    line._linewidth = 2.0/(0.1+abs(xF(tA)-L0))
    Kl.set_data(tpts,KF(tpts))
    Ul.set_data(tpts,UF(tpts))    
    return (pt,line,Kl,Ul)


anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=int((tmax-tmin)/dt), interval=50, 
                               blit=True)
plt.close()
anim
# plt.show()